<a href="https://colab.research.google.com/github/Nithyarajoman/Machine-learning-Trial/blob/main/Library_Functions_MAC/Data-Calculations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models

In [3]:
# MNIST Fashion dataset for the example cnn model
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float')/255
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1).astype('float')/255

In [6]:
# ImageNet classification dataset for ResNet50
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data() #Size is (, 32,32,3)

169001437/169001437 [==============================] - 2s 0us/step


In [57]:
# Create Model
model_hierarchy = models.Sequential(
    [        
        layers.Conv2D(32, kernel_size=(3, 3),
                      input_shape=(28, 28, 1),
                      strides=(2,2), padding="same",
                      use_bias=False),
        layers.BatchNormalization(),
        layers.Activation(activation="relu"),
        #layers.MaxPooling2D(pool_size=(2, 2)),
        models.Sequential([
            layers.Conv2D(64, kernel_size=(3, 3),
                          strides=(2,2), padding="valid",
                          use_bias=False),
            layers.BatchNormalization(),
            layers.Activation(activation="relu")
            ]),            
        #layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(10, activation='softmax')
    ]
)

In [60]:
# Train
model_hierarchy.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model_hierarchy.fit(x=x_train, y=y_train, epochs=5, batch_size=128, validation_split=0.1 )
score = model_hierarchy.evaluate(x_test, y_test, verbose=0)

Epoch 1/5
422/422 [==============================] - 20s 45ms/step - loss: 0.5800 - accuracy: 0.7937 - val_loss: 0.5510 - val_accuracy: 0.8317
Epoch 2/5
422/422 [==============================] - 20s 48ms/step - loss: 0.3939 - accuracy: 0.8582 - val_loss: 0.3213 - val_accuracy: 0.8850
Epoch 3/5
422/422 [==============================] - 19s 44ms/step - loss: 0.3497 - accuracy: 0.8736 - val_loss: 0.3065 - val_accuracy: 0.8873
Epoch 4/5
422/422 [==============================] - 19s 45ms/step - loss: 0.3220 - accuracy: 0.8828 - val_loss: 0.2867 - val_accuracy: 0.8935
Epoch 5/5
422/422 [==============================] - 19s 44ms/step - loss: 0.3041 - accuracy: 0.8895 - val_loss: 0.2881 - val_accuracy: 0.8927


In [7]:
# Function to calculate return MAC from each layer

def mac_each_layer(l):
    o_shape, i_shape, strides, ks, filters = ['', '', ''], ['', '', ''], [1, 1], [0, 0], [0, 0]        
    macc = 0
    
    if ('Dense' in str(l)):
        i_shape = l.input.shape[1:4].as_list()[0]
        o_shape = l.output.shape[1:4].as_list()
        macc = (o_shape[0] * i_shape)

    
    if ('Conv2D ' in str(l) and 'DepthwiseConv2D' not in str(l) and 'SeparableConv2D' not in str(l)):
        strides = l.strides
        ks = l.kernel_size
        filters = l.filters
        i_shape = l.input.get_shape()[1:4].as_list()
        o_shape = l.output.get_shape()[1:4].as_list()

        if (filters == None):
            filters = i_shape[2]

        macc = ((filters * ks[0] * ks[1] * i_shape[2]) * (
                (i_shape[0] / strides[0]) * (i_shape[1] / strides[1])))

    if ('Conv2D ' in str(l) and 'DepthwiseConv2D' in str(l) and 'SeparableConv2D' not in str(l)):
        strides = l.strides
        ks = l.kernel_size
        filters = l.filters
        i_shape = l.input.get_shape()[1:4].as_list()
        o_shape = l.output.get_shape()[1:4].as_list()

        if (filters == None):
            filters = i_shape[2]

        macc = ((ks[0] * ks[1] * i_shape[2]) * ((i_shape[0] / strides[0]) * (i_shape[1] / strides[1])))

    return macc

In [8]:
# Function to pass layers to the calculating function and to get the macc of each layer

def MAC_summary(model):
  print('%25s | %6s' % ('Layer Name', 'MACs'))
  print('*' * 100)
  t_macc = 0

  for l in model.layers:
    if (type(l) in [models.Sequential, keras.Model]):
      for ib_layer in l.layers:
        mac_obtained = mac_each_layer(ib_layer)
        print('%25s(%5s) | %5.4f' % (
                l.name,ib_layer.name, mac_obtained ))
        t_macc += mac_obtained  

    else:
      mac_obtained = mac_each_layer(l)
      print('%25s | %5.4f' % (
                l.name, mac_obtained ))
      
    t_macc += mac_obtained
  print('\nTotal MACs operations in the complete model: %10.8f\n' % (t_macc))
  return

In [9]:
# Function to check if a layer is model or not
def is_hier_layer(layer):
    "Finds if layer is actually a model instead of a single layer"
    return type(layer) in [models.Sequential, keras.Model]


# Function to add communicated data to the metrics
def add_metrics(m2):
    "Annotates recursiverly a (non-compiled) model with metrics that calculate probability of non-zero activity"
    prefix_name = m2.name + "_"
    for ll in m2.layers:
        if is_hier_layer(ll):
            add_metrics(ll)
        else:
            my_metric = tf.reduce_sum( tf.math.count_nonzero(ll.output, dtype=tf.int32) / tf.size( ll.output)  )
            name=prefix_name + ll.name
            m2.add_metric(my_metric, name=name)
    return


In [10]:
# Function to print the effective data communicated 

def data_summary(model):

  # Function to add communicated data to the metrics
  def add_metrics(m2):
    "Annotates recursiverly a (non-compiled) model with metrics that calculate probability of non-zero activity"
    prefix_name = m2.name + "_"
    for ll in m2.layers:
        if is_hier_layer(ll):
            add_metrics(ll)
        else:
            my_metric = tf.reduce_sum( tf.math.count_nonzero(ll.output, dtype=tf.int32) / tf.size( ll.output)  )
            name=prefix_name + ll.name
            m2.add_metric(my_metric, name=name)
    return

  # Function to check if a layer is model or not
  def is_hier_layer(layer):
      "Finds if layer is actually a model instead of a single layer"
      return type(layer) in [models.Sequential, keras.Model]

  # Clone the model. Note that it is not compiled!
  m2 = keras.models.clone_model(model)
  add_metrics(m2)
  
  # Compiling the cloned model and taking the inference
  m2.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  aux = m2.evaluate(x_test[0:1000], y_test[0:1000]) 
  
  print('%40s | %20s' % ('Metrics and Layer Name', 'Effective data communicated'))
  print("*" * 150)
  for l in m2.metrics:
    print('%40s | %5.4f' % (l.name, l.result().numpy()))
  return

In [ ]:
# MAC calculation for sample cnn
MAC_summary(model_hierarchy)

               Layer Name |   MACs
****************************************************************************************************
                conv2d_10 | 56448.0000
   batch_normalization_10 | 0.0000
            activation_10 | 0.0000
            sequential_10(conv2d_11) | 903168.0000
            sequential_10(batch_normalization_11) | 0.0000
            sequential_10(activation_11) | 0.0000
                flatten_5 | 0.0000
                dropout_5 | 0.0000
                  dense_5 | 23040.0000

Total MACs operations in the complete model: 982656.00000000



In [61]:
# Effective calculation for sample cnn
data_summary(model_hierarchy)

32/32 [==============================] - 1s 8ms/step - loss: 2.3007 - accuracy: 0.1150 - sequential_3_conv2d_1: 0.8909 - sequential_3_batch_normalization_4: 0.8909 - sequential_3_activation_1: 0.4174 - sequential_4_conv2d: 0.6556 - sequential_4_batch_normalization_3: 0.6556 - sequential_4_activation: 0.2935 - sequential_4_flatten: 0.4174 - sequential_4_dropout_3: 0.4174 - sequential_4_dense_6: 1.0000
                  Metrics and Layer Name | Effective data communicated
******************************************************************************************************************************************************
                                    loss | 2.3007
                                accuracy | 0.1150
                   sequential_3_conv2d_1 | 0.8909
      sequential_3_batch_normalization_4 | 0.8909
               sequential_3_activation_1 | 0.4174
                     sequential_4_conv2d | 0.6556
      sequential_4_batch_normalization_3 | 0.6556
                 sequent

In [1]:
# Calculations for Resnet18 CNN model
pip install image-classifiers
from tensorflow.keras.applications import imagenet_utils
from classification_models.keras import Classifiers

ResNet18, preprocess_input = Classifiers.get('resnet18')
model_ResNet18 = ResNet18(include_top=False,
                        input_tensor=None,
                        input_shape=(
                            32, 32, 3),
                        pooling=None)

In [11]:
MAC_summary(model_ResNet18)

               Layer Name |   MACs
****************************************************************************************************
                     data | 0.0000
                  bn_data | 0.0000
        zero_padding2d_18 | 0.0000
                    conv0 | 3396288.0000
                      bn0 | 0.0000
                    relu0 | 0.0000
        zero_padding2d_19 | 0.0000
                 pooling0 | 0.0000
         stage1_unit1_bn1 | 0.0000
       stage1_unit1_relu1 | 0.0000
        zero_padding2d_20 | 0.0000
       stage1_unit1_conv1 | 3686400.0000
         stage1_unit1_bn2 | 0.0000
       stage1_unit1_relu2 | 0.0000
        zero_padding2d_21 | 0.0000
       stage1_unit1_conv2 | 3686400.0000
          stage1_unit1_sc | 262144.0000
                    add_8 | 0.0000
         stage1_unit2_bn1 | 0.0000
       stage1_unit2_relu1 | 0.0000
        zero_padding2d_22 | 0.0000
       stage1_unit2_conv1 | 3686400.0000
         stage1_unit2_bn2 | 0.0000
       stage1_unit2_relu2 | 0.

In [12]:
data_summary(model_ResNet18)

32/32 [==============================] - 11s 113ms/step - loss: 12.9790 - accuracy: 0.0026 - model_1_data: 0.9966 - model_1_bn_data: 0.9966 - model_1_zero_padding2d_18: 0.7067 - model_1_conv0: 0.9997 - model_1_bn0: 0.9997 - model_1_relu0: 0.4305 - model_1_zero_padding2d_19: 0.3402 - model_1_pooling0: 0.6188 - model_1_stage1_unit1_bn1: 0.6188 - model_1_stage1_unit1_relu1: 0.6188 - model_1_zero_padding2d_20: 0.3960 - model_1_stage1_unit1_conv1: 1.0000 - model_1_stage1_unit1_bn2: 1.0000 - model_1_stage1_unit1_relu2: 0.4869 - model_1_zero_padding2d_21: 0.3116 - model_1_stage1_unit1_conv2: 1.0000 - model_1_stage1_unit1_sc: 0.9999 - model_1_add_8: 1.0000 - model_1_stage1_unit2_bn1: 1.0000 - model_1_stage1_unit2_relu1: 0.4433 - model_1_zero_padding2d_22: 0.2837 - model_1_stage1_unit2_conv1: 1.0000 - model_1_stage1_unit2_bn2: 1.0000 - model_1_stage1_unit2_relu2: 0.4105 - model_1_zero_padding2d_23: 0.2627 - model_1_stage1_unit2_conv2: 1.0000 - model_1_add_9: 1.0000 - model_1_stage2_unit1_bn1: 1

In [53]:
# Calculations for Resnet50 CNN model
from tensorflow.keras.applications.resnet50 import ResNet50
model_Resnet50 = ResNet50(include_top=False,
                        weights='imagenet',
                        input_shape=(32, 32, 3),
                        pooling=None)

In [54]:
MAC_summary(model_Resnet50)

               Layer Name |   MACs
****************************************************************************************************
                  input_5 | 0.0000
                conv1_pad | 0.0000
               conv1_conv | 3396288.0000
                 conv1_bn | 0.0000
               conv1_relu | 0.0000
                pool1_pad | 0.0000
               pool1_pool | 0.0000
      conv2_block1_1_conv | 262144.0000
        conv2_block1_1_bn | 0.0000
      conv2_block1_1_relu | 0.0000
      conv2_block1_2_conv | 2359296.0000
        conv2_block1_2_bn | 0.0000
      conv2_block1_2_relu | 0.0000
      conv2_block1_0_conv | 1048576.0000
      conv2_block1_3_conv | 1048576.0000
        conv2_block1_0_bn | 0.0000
        conv2_block1_3_bn | 0.0000
         conv2_block1_add | 0.0000
         conv2_block1_out | 0.0000
      conv2_block2_1_conv | 1048576.0000
        conv2_block2_1_bn | 0.0000
      conv2_block2_1_relu | 0.0000
      conv2_block2_2_conv | 2359296.0000
        conv2_bloc

In [55]:
data_summary(model_Resnet50)

32/32 [==============================] - 23s 248ms/step - loss: 11.4274 - accuracy: 0.0000e+00 - resnet50_input_5: 0.9966 - resnet50_conv1_pad: 0.7067 - resnet50_conv1_conv: 0.9997 - resnet50_conv1_bn: 0.9997 - resnet50_conv1_relu: 0.5794 - resnet50_pool1_pad: 0.4578 - resnet50_pool1_pool: 0.7660 - resnet50_conv2_block1_1_conv: 0.9999 - resnet50_conv2_block1_1_bn: 0.9999 - resnet50_conv2_block1_1_relu: 0.4261 - resnet50_conv2_block1_2_conv: 1.0000 - resnet50_conv2_block1_2_bn: 1.0000 - resnet50_conv2_block1_2_relu: 0.5328 - resnet50_conv2_block1_0_conv: 0.9999 - resnet50_conv2_block1_3_conv: 1.0000 - resnet50_conv2_block1_0_bn: 0.9999 - resnet50_conv2_block1_3_bn: 1.0000 - resnet50_conv2_block1_add: 1.0000 - resnet50_conv2_block1_out: 0.4614 - resnet50_conv2_block2_1_conv: 1.0000 - resnet50_conv2_block2_1_bn: 1.0000 - resnet50_conv2_block2_1_relu: 0.5140 - resnet50_conv2_block2_2_conv: 1.0000 - resnet50_conv2_block2_2_bn: 1.0000 - resnet50_conv2_block2_2_relu: 0.5216 - resnet50_conv2_b